In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import api_key
gmaps.configure(api_key=api_key)

In [8]:
# read the cities.csv to create dataframe
cities = pd.read_csv("../WeatherPy/output/CityWeatherData.csv", encoding="utf-8")
cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,ushuaia,40,AR,1600876514,43,-54.80,-68.30,51.80,19.46
1,busselton,0,AU,1600876514,52,-33.65,115.33,59.00,14.09
2,albany,40,AU,1600876514,93,-35.02,117.89,50.00,9.17
3,kamloops,90,CA,1600876514,62,50.67,-120.32,59.00,13.87
4,bengkulu,95,ID,1600876515,89,-3.80,102.27,78.66,6.38


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

fig = gmaps.figure()  # create new gmap figure
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, 
                                 max_intensity=maxhumidity,point_radius=3)  # create heatmap layer
fig.add_layer(heat_layer)  # add heat layer to cover map
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
city_df = cities[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0)]
city_df = city_df.dropna(how='any')  # drop any null values 
city_df = city_df.reset_index()  # reset index column
city_df.drop(["index"], axis=1, inplace=True)  # remove old index column
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed
0,nouadhibou,0,MR,1600876584,74,20.93,-17.03,78.80,17.22
1,bam,0,IR,1600876584,20,29.11,58.36,73.71,10.54
2,chitral,0,PK,1600876654,27,35.85,71.79,70.99,1.97
3,guerrero negro,0,MX,1600876720,56,27.98,-114.06,78.55,7.07
4,mount isa,0,AU,1600876723,26,-20.73,139.50,71.60,10.29


### Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotels = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i in range(len(city_df)):
    lat = city_df.loc[i]['Lat']
    lng = city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": api_key
    }
    
    try:
        response = requests.get(base_url, params=params)
        jsn = response.json()
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")

city_df["Hotel Name"] = hotels
city_df = city_df.dropna(how='any')
city_df.tail()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Windspeed,Hotel Name
14,kharan,0,PK,1600876998,17,28.58,65.42,78.76,3.74,
15,dahuk,0,IQ,1600877133,25,36.87,42.99,76.03,2.51,
16,tabas,0,IR,1600877135,17,33.60,56.92,71.91,7.70,
17,pahrump,0,US,1600877202,24,36.21,-115.98,73.99,4.38,
18,tukrah,0,LY,1600877269,55,32.53,20.58,79.16,2.30,


In [18]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in city_df.iterrows()]
locations = city_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations)  # create markers on the hotel locations
fig.add_layer(markers)  # add layer of markers to figure
print(fig)

Figure(layout=FigureLayout(height='420px'))
